In [1]:
!pip install biopython numba pandas numpy
!pip install blosum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
from numba import jit
import multiprocessing as mp
from functools import partial
import time
from Bio.Align import substitution_matrices

In [3]:
def make_blosum62() -> Tuple[np.ndarray, Dict[str, int], str]:
    amino_acids = "ARNDCQEGHILKMFPSTWYV"
    mat = substitution_matrices.load("BLOSUM62")
    arr = np.zeros((20, 20), dtype=np.float32)
    for i, a1 in enumerate(amino_acids):
        for j, a2 in enumerate(amino_acids):
            arr[i, j] = float(mat[(a1, a2)])
    aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}
    return arr, aa_to_idx, amino_acids

def encode_sequence(seq: str, aa_to_idx: Dict[str,int]) -> np.ndarray:
    seq = seq.upper()
    out = np.zeros(len(seq), dtype=np.int8)
    for i, ch in enumerate(seq):
        out[i] = aa_to_idx.get(ch, 0)
    return out

In [4]:
def read_fasta(filename: str):
    records = []
    with open(filename, "r") as f:
        header = None
        seq_chunks = []
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header is not None:
                    records.append((header, "".join(seq_chunks).upper()))
                header = line[1:].strip()
                seq_chunks = []
            else:
                seq_chunks.append(line)
        if header is not None:
            records.append((header, "".join(seq_chunks).upper()))
    return records

In [5]:
@jit(nopython=True)
def smith_waterman_dp(seq1_enc: np.ndarray,
                      seq2_enc: np.ndarray,
                      blosum: np.ndarray,
                      gap_open: float,
                      gap_extend: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float, int, int]:
    n, m = len(seq1_enc), len(seq2_enc)
    M = np.full((n+1, m+1), -np.inf, dtype=np.float32)
    X = np.full((n+1, m+1), -np.inf, dtype=np.float32)
    Y = np.full((n+1, m+1), -np.inf, dtype=np.float32)

    for j in range(m+1):
        M[0, j] = 0.0
        X[0, j] = -np.inf
        Y[0, j] = 0.0
    for i in range(n+1):
        M[i, 0] = 0.0
        X[i, 0] = 0.0
        Y[i, 0] = -np.inf

    best = 0.0
    bi = 0
    bj = 0

    for i in range(1, n+1):
        ai = seq1_enc[i-1]
        for j in range(1, m+1):
            bj_idx = seq2_enc[j-1]
            s = blosum[ai, bj_idx]

            X[i, j] = max(X[i-1, j] - gap_extend, M[i-1, j] - gap_open)
            Y[i, j] = max(Y[i, j-1] - gap_extend, M[i, j-1] - gap_open)

            M[i, j] = max(0.0, M[i-1, j-1] + s, X[i, j], Y[i, j])

            if M[i, j] > best:
                best = M[i, j]
                bi = i
                bj = j

    return M, X, Y, float(best), bi, bj

In [6]:
def sw_traceback(seq1: str, seq2: str,
                 M: np.ndarray, X: np.ndarray, Y: np.ndarray,
                 gap_open: float, gap_extend: float,
                 blosum: np.ndarray,
                 aa_to_idx: Dict[str,int],
                 start_i: int, start_j: int) -> Tuple[str, str]:
    i, j = start_i, start_j
    aln1, aln2 = [], []

    def s(a, b):
        return float(blosum[aa_to_idx.get(a, 0), aa_to_idx.get(b, 0)])

    while i > 0 and j > 0:
        if M[i, j] <= 0:
            break

        if abs(M[i, j] - (M[i-1, j-1] + s(seq1[i-1], seq2[j-1]))) < 1e-5:
            aln1.append(seq1[i-1])
            aln2.append(seq2[j-1])
            i -= 1
            j -= 1
            continue

        if abs(M[i, j] - X[i, j]) < 1e-5:
            aln1.append(seq1[i-1]); aln2.append('-'); i -= 1; continue
        if abs(M[i, j] - Y[i, j]) < 1e-5:
            aln1.append('-'); aln2.append(seq2[j-1]); j -= 1; continue

        aln1.append(seq1[i-1]); aln2.append(seq2[j-1]); i -= 1; j -= 1

    return ''.join(reversed(aln1)), ''.join(reversed(aln2))

In [7]:
class SmithWatermanAligner:
    def __init__(self, gap_open: float = 11.0, gap_extend: float = 1.0):
        self.gap_open = float(gap_open)
        self.gap_extend = float(gap_extend)
        self.blosum, self.aa_to_idx, self.aa_order = make_blosum62()

    def align(self, seq1: str, seq2: str) -> Dict[str, object]:
        s1, s2 = seq1.upper(), seq2.upper()
        e1, e2 = encode_sequence(s1, self.aa_to_idx), encode_sequence(s2, self.aa_to_idx)

        M, X, Y, best, bi, bj = smith_waterman_dp(e1, e2, self.blosum, self.gap_open, self.gap_extend)
        aln1, aln2 = sw_traceback(s1, s2, M, X, Y, self.gap_open, self.gap_extend, self.blosum, self.aa_to_idx, bi, bj)

        matches = sum(1 for a, b in zip(aln1, aln2) if a == b and a != '-' and b != '-')
        mismatches = sum(1 for a, b in zip(aln1, aln2) if a != b and a != '-' and b != '-')
        g1, g2 = aln1.count('-'), aln2.count('-')
        aligned_positions = matches + mismatches
        identity = (matches / aligned_positions) if aligned_positions > 0 else 0.0
        distance = 1.0 - identity

        return {
            "alignment_score": float(best),
            "seq1_length": len(seq1),
            "seq2_length": len(seq2),
            "aligned_length": len(aln1),
            "aligned_positions": aligned_positions,
            "matches": matches,
            "mismatches": mismatches,
            "gaps_seq1": g1,
            "gaps_seq2": g2,
            "identity": identity,
            "distance": distance,
            "alignment1": aln1,
            "alignment2": aln2
        }

In [12]:
def pairs_from_fasta_records(records: List[Tuple[str,str]]):
    N = len(records)
    pairs = []
    for i in range(N):
        h1, s1 = records[i]
        for j in range(i+1, N):
            h2, s2 = records[j]
            pairs.append((i, j, s1, s2, h1, h2))
    return pairs

def parse_header(header: str):
    """
    Parse a header like:
    '2328020901 Antechinus flavipes | Dasyuromorphia | Marsupials'
    into (protein_id, species, order, superorder).
    """
    if header is None:
        return "", "", "", ""

    parts = [p.strip() for p in header.split("|")]
    if len(parts) != 3:
        # Fallback if the format is unexpected
        return header.strip(), "", "", ""

    left, order, superorder = parts
    left = left.strip()

    # First token = protein_id, everything after = species name
    first_space = left.find(" ")
    if first_space == -1:
        protein_id = left
        species = ""
    else:
        protein_id = left[:first_space].strip()
        species = left[first_space + 1:].strip()

    return protein_id, species, order, superorder


def worker_align_fasta(pair, gap_open, gap_extend, blosum, aa_to_idx):
    i, j, s1, s2, h1, h2 = pair

    # --- Smith–Waterman core (same as before) ---
    e1 = encode_sequence(s1, aa_to_idx)
    e2 = encode_sequence(s2, aa_to_idx)
    M, X, Y, best, bi, bj = smith_waterman_dp(e1, e2, blosum, gap_open, gap_extend)
    aln1, aln2 = sw_traceback(
        s1, s2, M, X, Y,
        gap_open, gap_extend,
        blosum, aa_to_idx,
        bi, bj
    )

    # ---------------------------------------------------------
    # REVISED METRIC LOGIC (Consistent with Global/SemiGlobal)
    # ---------------------------------------------------------

    # Recalculate basic counts
    # (Using strict loop or count methods ensures we catch everything correctly)
    matches = 0
    mismatches = 0
    gaps_seq1 = 0
    gaps_seq2 = 0

    for k in range(len(aln1)):
        if aln1[k] == '-':
            gaps_seq1 += 1
        elif aln2[k] == '-':
            gaps_seq2 += 1
        elif aln1[k] == aln2[k]:
            matches += 1
        else:
            mismatches += 1

    # Original unaligned lengths (passed in as s1, s2)
    n = len(s1)
    m = len(s2)

    # Calculate Normalization Factor (Min Original Length)
    # This is CRITICAL for Smith-Waterman.
    # Without this, short perfect motifs get distance=0.0.
    # With this, they get penalized for not covering the full sequence.
    min_original_len = min(n, m)

    aligned_length = len(aln1)
    aligned_positions = matches + mismatches

    if min_original_len > 0:
        # Identity relative to the SHORTER sequence
        identity = matches / min_original_len
        distance = 1.0 - identity
    else:
        identity = 0.0
        distance = 1.0

    # Sanity check: Distance cannot be negative
    distance = max(0.0, distance)

    # ---------------------------------------------------------
    # END METRIC LOGIC
    # ---------------------------------------------------------

    # --- NEW: parse headers into metadata like code1 ---
    pid1, species1, order1, superorder1 = parse_header(h1)
    pid2, species2, order2, superorder2 = parse_header(h2)

    # --- Return in the SAME FORMAT as code1 ---
    return {
        # metadata
        "protein_id_1": pid1,
        "protein_id_2": pid2,
        "species_1": species1,
        "species_2": species2,
        "order_1": order1,
        "order_2": order2,
        "superorder_1": superorder1,
        "superorder_2": superorder2,

        # alignment settings
        "matrix": "BLOSUM62",          # you are using BLOSUM62 for SW
        "gap_open": gap_open,
        "gap_extend": gap_extend,

        # alignment stats – renamed to match code1
        "score": float(best),          # alignment_score -> score
        "aln1": aln1,                  # alignment1 -> aln1
        "aln2": aln2,                  # alignment2 -> aln2
        "identity": identity,
        "distance": distance,
        "alignment_length": aligned_length,          # aligned_length -> alignment_length
        "aligned_non_gap_positions": aligned_positions,
        "matches": matches,

        # optional extras (fine to keep)
        "mismatches": mismatches,
        "gaps_seq1": gaps_seq1,
        "gaps_seq2": gaps_seq2,
        "seq1_length": n,
        "seq2_length": m,
    }



def run_all_sw_fasta(fasta_path, processes=None, chunk=200, gap_open=11, gap_extend=1):
    blosum, aa_to_idx, _ = make_blosum62()
    records = read_fasta(fasta_path)
    print("Loaded", len(records), "sequences from FASTA")

    pairs = pairs_from_fasta_records(records)
    print("Total pairs:", len(pairs))

    if processes is None:
        processes = max(1, mp.cpu_count()-1)

    start = time.time()
    results = []
    with mp.Pool(processes=processes) as pool:
        func = partial(worker_align_fasta,
                       gap_open=gap_open, gap_extend=gap_extend,
                       blosum=blosum, aa_to_idx=aa_to_idx)
        for k, out in enumerate(pool.imap_unordered(func, pairs, chunksize=chunk), 1):
            results.append(out)
            if k % 1000 == 0:
                rate = k / max(time.time()-start, 1e-6)
                rem = (len(pairs)-k) / max(rate, 1e-6)
                print(f"{k:,}/{len(pairs):,}  ~{rem:.0f}s remaining")

    print(f"Done {len(results)} pairs in {time.time()-start:.1f}s")
    return pd.DataFrame(results)



In [13]:
fasta_path = "/content/hemoglobin_209_species_final_fasta.fasta"
out_df = run_all_sw_fasta(fasta_path, processes=4, chunk=200, gap_open=11, gap_extend=1)
out_csv = "/content/sample_data/smith_waterman_alignments.csv"
out_df.to_csv(out_csv, index=False)
print("Saved:", out_csv, "Rows:", len(out_df))

Loaded 209 sequences from FASTA
Total pairs: 21736
1,000/21,736  ~116s remaining
2,000/21,736  ~63s remaining
3,000/21,736  ~47s remaining
4,000/21,736  ~37s remaining
5,000/21,736  ~31s remaining
6,000/21,736  ~26s remaining
7,000/21,736  ~22s remaining
8,000/21,736  ~19s remaining
9,000/21,736  ~16s remaining
10,000/21,736  ~14s remaining
11,000/21,736  ~12s remaining
12,000/21,736  ~10s remaining
13,000/21,736  ~9s remaining
14,000/21,736  ~8s remaining
15,000/21,736  ~6s remaining
16,000/21,736  ~5s remaining
17,000/21,736  ~4s remaining
18,000/21,736  ~3s remaining
19,000/21,736  ~2s remaining
20,000/21,736  ~1s remaining
21,000/21,736  ~1s remaining
Done 21736 pairs in 17.2s
Saved: /content/sample_data/smith_waterman_alignments.csv Rows: 21736


In [14]:
df = pd.read_csv('/content/sample_data/smith_waterman_alignments.csv')

df.head()

,protein_id_1,protein_id_2,species_1,species_2,order_1,order_2,superorder_1,superorder_2,matrix,gap_open,...,identity,distance,alignment_length,aligned_non_gap_positions,matches,mismatches,gaps_seq1,gaps_seq2,seq1_length,seq2_length
0,2328020901,122394,Antechinus flavipes,Dasyurus viverrinus,Dasyuromorphia,Dasyuromorphia,Marsupials,Marsupials,BLOSUM62,11,...,0.053097,0.946903,23,23,6,17,0,0,113,142
1,2328020901,395515590,Antechinus flavipes,Sarcophilus harrisii,Dasyuromorphia,Dasyuromorphia,Marsupials,Marsupials,BLOSUM62,11,...,0.106195,0.893805,47,44,12,32,0,3,113,142
2,2328020901,2990525618,Antechinus flavipes,Sminthopsis crassicaudata,Dasyuromorphia,Dasyuromorphia,Marsupials,Marsupials,BLOSUM62,11,...,0.929204,0.070796,113,113,105,8,0,0,113,113
3,2328020901,122395,Antechinus flavipes,Didelphis virginiana,Dasyuromorphia,Didelphimorphia,Marsupials,Marsupials,BLOSUM62,11,...,0.061947,0.938053,31,31,7,24,0,0,113,141
4,2328020901,2119467502,Antechinus flavipes,Gracilinanus agilis,Dasyuromorphia,Didelphimorphia,Marsupials,Marsupials,BLOSUM62,11,...,0.061947,0.938053,19,19,7,12,0,0,113,142


In [11]:
# Validation cell: independent Smith–Waterman spot-check with Biopython

import pandas as pd
import numpy as np
from Bio import pairwise2
from Bio.Align import substitution_matrices
from Bio import SeqIO

# ---- Paths (adjust if needed) ----
sw_csv = "/content/sample_data/smith_waterman_alignments.csv"   # your SW CSV (code2 output in code1 format)
fasta_path = "/content/hemoglobin_209_species_final_fasta.fasta"

# ---- Load SW results and FASTA sequences ----
sw = pd.read_csv(sw_csv)

records = list(SeqIO.parse(fasta_path, "fasta"))
# Map from protein_id (FASTA id) -> sequence
id_to_seq = {str(rec.id): str(rec.seq).upper() for rec in records}

print("Loaded SW rows:", len(sw))
print("Loaded FASTA records:", len(records))

# ---- Sample some rows to validate ----
np.random.seed(42)
k = min(50, len(sw))   # up to 50 random alignments
sample = sw.sample(n=k, replace=False).reset_index(drop=True)

# ---- Biopython SW configuration ----
gap_open, gap_extend = 11, 1
mat = substitution_matrices.load("BLOSUM62")

def sw_bio_metrics(s1: str, s2: str):
    """
    Compute Smith–Waterman alignment with Biopython and
    return (score, identity) to compare with our implementation.
    """
    aln = pairwise2.align.localds(
        s1, s2,
        mat,
        -gap_open,    # gap open penalty
        -gap_extend,  # gap extend penalty
        one_alignment_only=True
    )
    if not aln:
        return np.nan, np.nan

    a = aln[0]
    a1, a2 = a.seqA, a.seqB

    matches = sum(1 for x, y in zip(a1, a2) if x == y and x != '-' and y != '-')
    subs    = sum(1 for x, y in zip(a1, a2) if x != y and x != '-' and y != '-')
    ident   = matches / max(1, matches + subs)

    return float(a.score), float(ident)

# ---- Compute Biopython scores/identity for the sample ----
bio_scores, bio_idents = [], []

for _, r in sample.iterrows():
    pid1 = str(r["protein_id_1"])
    pid2 = str(r["protein_id_2"])

    # Look up sequences by protein_id from FASTA
    s1 = id_to_seq[pid1]
    s2 = id_to_seq[pid2]

    sc, idt = sw_bio_metrics(s1, s2)
    bio_scores.append(sc)
    bio_idents.append(idt)

sample["score_bio"] = bio_scores
sample["identity_bio"] = bio_idents

# ---- Compare our implementation vs Biopython ----
# Note: our SW CSV now uses `score` (not `alignment_score`)
print(sample[["score", "score_bio", "identity", "identity_bio"]].head(10))

print("Score Pearson r:", sample["score"].corr(sample["score_bio"]))
print("Identity MAE:", (sample["identity"] - sample["identity_bio"]).abs().mean())
print("Identity max abs diff:", (sample["identity"] - sample["identity_bio"]).abs().max())


/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Loaded SW rows: 21736
Loaded FASTA records: 209
   score  score_bio  identity  identity_bio
0  545.0      545.0  0.739437      0.739437
1  533.0      533.0  0.718310      0.718310
2  549.0      549.0  0.751773      0.751773
3  619.0      619.0  0.836879      0.836879
4  582.0      582.0  0.795775      0.795775
5  608.0      608.0  0.822695      0.822695
6  624.0      624.0  0.836879      0.836879
7   24.0       24.0  0.344828      0.120690
8  630.0      630.0  0.852113      0.852113
9  353.0      353.0  0.666667      0.654545
Score Pearson r: 1.0
Identity MAE: 0.04883393608499984
Identity max abs diff: 0.31028368794326244
